In [9]:
from haystack import component
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from typing import List, Dict
from haystack.utils import Secret

In [10]:
MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q8_0"
# Инициализация генератора
llm = OpenAIChatGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={"temperature": 0.7}
)

In [11]:
@component
class QueryDecomposerLLM:
    """
    Через LLM определяет: нужно ли разбивать сложный запрос на под-вопросы,
    и если да — генерирует эти суб‑вопросы.
    """
    def __init__(self, generator: OpenAIChatGenerator):
        self.generator = generator

        # 1️⃣ Шаблон оценки необходимости разбиения (few‑shot на сложные запросы)
        self.check_template = """
Ты — эксперт по обработке поисковых запросов.
Если запрос действительно сложный и стоит разбить его на несколько составляющих под‑вопросов, 
ответь 'true'. Иначе — 'false'.

Примеры:
Запрос: "Расскажи, пожалуйста, о новых политиках отпуска и сколько дней теперь положено?"
Ответ: true

Запрос: "У кого больше родственников, у Сансы или у Джейме?"
Ответ: true

Запрос: "Что такое GDPR?"
Ответ: false

Теперь оцени запрос:
Запрос: "{{ query }}"
Ответ:
""".strip()

        # 2️⃣ Шаблон генерации под‑вопросов (few‑shot)
        self.decomp_template = """
Ты — специалист по разбору сложных вопросов на логические части.
Выведи список коротких, независимых под‑вопросов (каждый на новой строке).

Примеры:
Оригинал: "Расскажи, пожалуйста, о новых политиках отпуска и сколько дней теперь положено?"
- Расскажи о новых политиках отпуска.
- Сколько дней отпуска теперь положено?

Оригинал: "У кого больше родственников, у Сансы или у Джейме?"
- Сколько братьев и сестер у Сансы?
- Сколько братьев и сестер у Джейме?

Теперь разбей запрос:
Оригинал: "{{ query }}"
""".strip()

    @component.output_types(subqueries=List[str])
    def run(self, query: str) -> Dict[str, List[str]]:
        # —————————————————————————––––––––––––––––––––––––
        # 1) Сначала проверяем: нужно ли разбивать
        # —————————————————————————––––––––––––––––––––––––
        check_prompt = self.check_template.replace("{{ query }}", query)
        check_msg    = ChatMessage.from_user(check_prompt)
        check_out    = self.generator.run([check_msg])
        decision_txt = check_out.get("replies", [])[0].text.strip().lower() if check_out.get("replies") else ""
        needs_split  = decision_txt.startswith("true")

        if not needs_split:
            return {"subqueries": [query]}

        # —————————————————————————––––––––––––––––––––––––
        # 2) Генерируем список под‑вопросов
        # —————————————————————————––––––––––––––––––––––––
        decomp_prompt = self.decomp_template.replace("{{ query }}", query)
        decomp_msg    = ChatMessage.from_user(decomp_prompt)
        decomp_out    = self.generator.run([decomp_msg])
        raw_text      = decomp_out.get("replies", [])[0].text

        # Разбираем по строкам, удаляем маркеры "-"
        parts = [line.strip("- ").strip() for line in raw_text.splitlines() if line.strip()]
        return {"subqueries": parts}

In [12]:
from haystack import Pipeline

# Собираем Pipeline
pipe = Pipeline()
pipe.add_component("decomposer", QueryDecomposerLLM(generator=llm))

# Тестовый запуск
out = pipe.run({"query": "Расскажи о новых политиках отпуска и сколько дней теперь положено?"})
print(out["decomposer"]["subqueries"])

['Расскажи о новых правилах и политиках отпуска.', 'Сколько дней отпуска теперь положено?']


In [13]:
from typing import List, Dict
from haystack import component
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from pathlib import Path


# ──────────────────────────────────────────────────────────────────────────────
# 2) Список тестовых случаев
# ──────────────────────────────────────────────────────────────────────────────
test_cases = [
    {
        "query": "Привет",
        "should_split": False,
        "expected": ["Привет"]
    },
    {
        "query": "Как дела?",
        "should_split": False,
        "expected": ["Как дела?"]
    },
    {
        "query": "Что такое GDPR?",
        "should_split": False,
        "expected": ["Что такое GDPR?"]
    },
    {
        "query": "Расскажи, пожалуйста, о новых политиках отпуска и сколько дней теперь положено?",
        "should_split": True,
        "expected": [
            "Расскажи о новых политиках отпуска.",
            "Сколько дней отпуска теперь положено?"
        ]
    },
    {
        "query": "У кого больше родственников, у Сансы или у Джейме?",
        "should_split": True,
        "expected": [
            "Сколько братьев и сестёр у Сансы?",
            "Сколько братьев и сестёр у Джейме?"
        ]
    },
    {
        "query": "Где найти инструкцию по подаче заявления и форму отчёта о проделанной работе?",
        "should_split": True,
        "expected": [
            "Где найти инструкцию по подаче заявления?",
            "Где найти форму отчёта о проделанной работе?"
        ]
    },
    {
        "query": "Погода на завтра в Москве и Санкт-Петербурге",
        "should_split": True,
        "expected": [
            "Какая погода будет завтра в Москве?",
            "Какая погода будет завтра в Санкт-Петербурге?"
        ]
    },
    {
        "query": "Сколько километров между Парижем и Лондоном?",
        "should_split": False,
        "expected": ["Сколько километров между Парижем и Лондоном?"]
    },
    {
        "query": "Расписание матчей Лиги чемпионов и результаты прошлых игр",
        "should_split": True,
        "expected": [
            "Расписание матчей Лиги чемпионов.",
            "Каковы результаты прошлых игр Лиги чемпионов?"
        ]
    },
    {
        "query": "Спасибо, до встречи!",
        "should_split": False,
        "expected": ["Спасибо, до встречи!"]
    },
    {
        "query": "Какие новые функции в версии 2.12 и как они влияют на производительность?",
        "should_split": True,
        "expected": [
            "Какие новые функции в версии 2.12?",
            "Как эти функции влияют на производительность?"
        ]
    },
    {
        "query": "Кто выиграл Оскар в 2023 году?",
        "should_split": False,
        "expected": ["Кто выиграл Оскар в 2023 году?"]
    },
    {
        "query": "Опишите архитектуру микросервисов и примеры применяемых технологий",
        "should_split": True,
        "expected": [
            "Опишите архитектуру микросервисов.",
            "Приведите примеры применяемых технологий."
        ]
    },
    {
        "query": "Какой курс доллара и евро к рублю сейчас?",
        "should_split": True,
        "expected": [
            "Какой курс доллара к рублю сейчас?",
            "Какой курс евро к рублю сейчас?"
        ]
    },
    {
        "query": "Расписание поездов и билеты на поезд Москва — Санкт-Петербург",
        "should_split": True,
        "expected": [
            "Расписание поездов Москва — Санкт-Петербург.",
            "Как купить билеты на поезд Москва — Санкт-Петербург?"
        ]
    },
    {
        "query": "Чем отличается искусство итальянского Ренессанса и барокко, и назови ключевых художников каждого периода",
        "should_split": True,
        "expected": [
            "Чем отличается искусство итальянского Ренессанса и барокко?",
            "Назови ключевых художников итальянского Ренессанса и барокко."
        ]
    },
    {
        "query": "Как устроена экономика Китая?",
        "should_split": False,
        "expected": ["Как устроена экономика Китая?"]
    },
    {
        "query": "Расписание и цены на авиабилеты, а также правила провоза багажа",
        "should_split": True,
        "expected": [
            "Каково расписание и цены на авиабилеты?",
            "Каковы правила провоза багажа?"
        ]
    },
    {
        "query": "Объясни принципы машинного обучения и дай примеры supervised и unsupervised алгоритмов",
        "should_split": True,
        "expected": [
            "Объясни принципы машинного обучения.",
            "Приведи примеры supervised алгоритмов.",
            "Приведи примеры unsupervised алгоритмов."
        ]
    },
    {
        "query": "Пока",
        "should_split": False,
        "expected": ["Пока"]
    },
]

# ──────────────────────────────────────────────────────────────────────────────
# 3) Запуск тестов, расчёт accuracy и печать разбитий
# ──────────────────────────────────────────────────────────────────────────────
decomposer = QueryDecomposerLLM(generator=llm)

total = len(test_cases)
correct = 0
mismatches = []

for tc in test_cases:
    out = decomposer.run(query=tc["query"])
    got = out["subqueries"]
    expect = tc["expected"]
    match = (got == expect)
    if match:
        correct += 1
    else:
        mismatches.append({
            "query": tc["query"],
            "expected": expect,
            "got": got
        })

accuracy = correct / total * 100
print(f"Decomposition Accuracy: {accuracy:.2f}% ({correct}/{total})\n")

if mismatches:
    print("Мismatched cases:")
    for m in mismatches:
        print(f"- Запрос: {m['query']}")
        print(f"  Ожидалось: {m['expected']}")
        print(f"  Получено:  {m['got']}\n")
else:
    print("Все запросы разбиты корректно.")


Decomposition Accuracy: 35.00% (7/20)

Мismatched cases:
- Запрос: Расскажи, пожалуйста, о новых политиках отпуска и сколько дней теперь положено?
  Ожидалось: ['Расскажи о новых политиках отпуска.', 'Сколько дней отпуска теперь положено?']
  Получено:  ['Что изменилось в политике предоставления отпусков?', 'Сколько дней отпуска положено сейчас?']

- Запрос: У кого больше родственников, у Сансы или у Джейме?
  Ожидалось: ['Сколько братьев и сестёр у Сансы?', 'Сколько братьев и сестёр у Джейме?']
  Получено:  ['Сколько родителей у Сансы?', 'Есть ли у Сансы братья/сестры? Если да, сколько?', 'Сколько родителей у Джейме?', 'Есть ли у Джейме братья/сестры? Если да, сколько?', 'Имеются ли иные родственники (тети, дяди) у Сансы?', 'Имеются ли иные родственники (тети, дяди) у Джейме?']

- Запрос: Где найти инструкцию по подаче заявления и форму отчёта о проделанной работе?
  Ожидалось: ['Где найти инструкцию по подаче заявления?', 'Где найти форму отчёта о проделанной работе?']
  Получено:  [